# 군집화

### 4. 텍스트 분류 실습 - 20 뉴스그룹 분류

* 텍스트 분류  
  1. 학습 데이터로 특정 문서의 분류를 학습  
  2. 모델 생성  
  3. 학습 모델로 다른 문서의 분류 예측  
* 텍스트 피처 벡터화 => 일반적으로 희소 행렬 형태  
  - 효과적인 분류 알고리즘:  로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등

#### 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
# filenames: 데이터 세트를 저장하는 디렉터리와 파일명

In [4]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 크래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 크래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
# 개별 데이터의 텍스트 구성
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 헤더(header), 푸터(footer) 정보가 target 클래스와 유사한 데이터를 갖는 경우가 많음  
   -> 피처 포함 시, 대부분 높은 예측 성능 나타냄   
   => 정확한 성능 평가를 위해 remove 파라미터로 **헤더, 푸터 등 제거**

In [11]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


#### 피처 벡터화 변환, 머신러닝 모델 학습/예측/평가

* 테스트 데이터 피처 벡터화 수행 시 유의할 점  
  * **학습 데이터**를 이용하여 fit( ) -> 테스트 데이터 변환 transform( )
  * fit_transform( ) 사용하면 안됨  
    * 학습 시 사용된 피처 개수와 예측 시 사용되는 피처 개수가 다르기 때문

##### Count Vectorization

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()한 CountVectorizer 객체로 테스트 데이터 피처 벡터화 변환
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [13]:
# 11314 개의 문서에서 피처 101631 개 추출함

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 로지스틱 회귀로 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)

pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression 예측 정확도는 0.603


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
# 예측 정확도는 약 0.603

##### TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF로 피처 벡터화 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# 학습 데이터로 fit()한 TF-IDF 객체로 테스트 데이터 피처 벡터화 변환
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 로지스틱 회귀로 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)

pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression 예측 정확도는 0.674


In [ ]:
# TF-IDF 의 정확도가 단순 카운트 기반 정확도보다 높음
# 텍스트와 문서가 많은 텍스트 분석에서 TF-IDF 벡터화의 예측 결과가 더 좋음

* 머신러닝 모델 성능 향상 방법  
  1. 최적의 ML 알고리즘 선택
  2. 최상의 피처 전처리 수행  

  => TF-IDF 에 다양한 파라미터 적용 후 에측 성능 비교하기

In [17]:
# stop words 필터링 추가, ngram을 기본 (1,1) -> (1,2)로 변경하여 피처 벡터화
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range=(1,2), max_df=300)  # max_df: 빈도수 높은 단어 제외, ngram_range = (1,2): 토큰화된 단어 1개씩, 순서대로 2개씩 묶음
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# 학습 데이터로 fit()한 TF-IDF 객체로 테스트 데이터 피처 벡터화 변환
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 로지스틱 회귀로 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)

pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Vectorized Logistic Regression 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))


TF-IDF Vectorized Logistic Regression 예측 정확도는 0.692


* 로지스틱 회귀 하이퍼 파라미터 최적화 수행

In [18]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트으로 설정.
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.701


In [19]:
# C:10 일때 가장 좋은 예측 성능
# 예측 정확도는 0.701로 향상됨

#### 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합

* 사이킷런 pipeline 사용  

  1. 직관적인 ML 모델 코드 생성
  2. 수행 시간 절약
    * 대용량 데이터의 피처 벡터화 결과를 저장하지 않고 바로 ML 알고리즘에 적용  
  * 피처 벡터화, ML 알고리즘 학습/예측 코드 작성 한번에 진행


In [21]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf 로 생성하는 Pipeline 생성
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),  # 피처 벡터화 (TF-IDF)
                     ('lr_clf', LogisticRegression(C=10))])  # ML 알고리즘 (로지스틱 회귀 )

# 별도의 TfidfVectorizer 객체의 fit(), transform() 과 로지스틱 회귀의 fit(), predict() 필요 없음 (파이프라인 사용)
# pipeline의 fit(), predict() 만으로 피처 벡터화, ML 학습/예측 한번에 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 로지스틱 회귀의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 로지스틱 회귀의 예측 정확도는 0.701


* Pipeline 기반 하이퍼 파라미터 튜닝  
  * GridSearchCV 클래스의 생성 파라미터로 Pipeline 입력  
    * TfidfVectorizer 파라미터, Logistic Regression 하이퍼 파라미터 모두 최적화
  * param_grid 입력 값 설정
    * 딕셔너리 형태 (Key, Value)
    * **Key**:  하이퍼 파라미터명이 객체 변수명과 결합됨  
  * 수행 시간이 길다


In [26]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf 로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
    ])

# Pipeline에 기술된 각각의 개체 변수에 언더바(_) 2개 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름, 값 설정
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1, 5, 10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7533152393023013
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.701


In [ ]:
# max_df: 700, ngram_range: (1,2)로 피처 벡터화
# C: 10 적용하여 로지스틱 회귀
# -> 가장 좋은 검증 세트 성능 수치 도출됨, 그러나 실제 예측 정확도는 0.701로 거의 향상되지 않음